In [ ]:
import asyncio

from pydantic import *
from aiokafka import AIOKafkaProducer

from fastkafka import FastKafka
from fastkafka.testing import Tester, ApacheKafkaBroker

In [ ]:
class TestMsg(BaseModel):
    msg: str = Field(...)


app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))
# app.benchmark_results["test"] = dict(count=0)


@app.consumes()
@app.benchmark(interval=1, sliding_window_size=5)
async def on_preprocessed_signals(msg: TestMsg):
    to_predictions(TestMsg(msg="prediction"))


@app.produces(max_buffer_size = 100_000)
@app.benchmark(interval=1, sliding_window_size=5)
def to_predictions(prediction: TestMsg, ) -> TestMsg:
    #     print(f"Sending prediction: {prediction}")
    return prediction


@app.consumes()
@app.benchmark(interval=1, sliding_window_size=5)
async def on_predictions(msg: TestMsg):
    pass

async with ApacheKafkaBroker(topics=["preprocessed_signals", "predictions"]) as broker:
    producer = AIOKafkaProducer(bootstrap_servers=broker)
    await producer.start()
    try:
        for _ in range(100_000):
            sent = await producer.send("preprocessed_signals", b'{"msg": "hello"}')
    finally:
        await producer.stop()
        
    print("*"*50+"Starting benchmark"+"*"*50)
    
    async with app:
        await asyncio.sleep(5)

print("ok")

[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
**************************************************Starting benchmark**************************************************
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._components.aiokafka_producer_manager: AIOKafkaProducerManager.start(): Entering...
[INFO] fastkafka._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting...
[INFO] fastkafka._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting send_stream
[INFO] fastkafka._components.aiokafka_producer_manager: AIOKafkaPr